In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from skimage import data
from skimage.filters import threshold_otsu
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, square
from skimage.color import label2rgb


In [2]:
from PIL import Image, ImageDraw, ImageOps
import numpy as np
import copy
import os
import uuid
from zipfile import ZipFile
import time

In [3]:
def show_image(data, transpose = False):
    if transpose == True:
        im = Image.fromarray(data.transpose())
    else:
        im = Image.fromarray(data)
    new_im = im.convert('RGB')
    new_im.show()

In [4]:
def rgb_to_grayscale(pixel):
    result = (0.2126 * pixel[0] + 0.7152 * pixel[1] + 0.0722 * pixel[2])
    return result

In [5]:
def convert_binary(data, thresh, b):
    if b == 1:
        converted = np.where(data < thresh, 255, 0)
    else:
        converted = np.where(data < thresh, 0, 255)
    replacements = np.unique(converted, return_counts = True)
    return converted, replacements

In [6]:
def save_images(imagePath, savePath, colours1, colours2, colours3, padding, show, thresh):
    img = Image.open(imagePath)

    #img = img.convert('RGB')
    image = np.array(img)
    image1 = copy.deepcopy(image)
    pixels = img.load()
    
    size = image.shape
    for i in range(0, size[0]):
        for j in range(0, size[1]):
            if image[i][j] in colours1:
                image[i][j] = 35    
            else:
                image[i][j] = 0
    
    cleared = clear_border(image)
    label_image = label(cleared)
    image_label_overlay = label2rgb(label_image, image=image, bg_label=0)
    
    rectangles = []
    if show == True:
        fig, ax = plt.subplots(figsize=(30, 18))
        ax.imshow(img)

    for region in regionprops(label_image):
        # take regions with large enough areas
        if region.area >= 100:
            
            minr, minc, maxr, maxc = region.bbox
            rectangles.append((minr, minc, maxr, maxc))
            
            if show == True:
                rect = mpatches.Rectangle((minc, minr), maxc - minc, maxr - minr,
                                      fill=False, edgecolor='red', linewidth=2)
                ax.add_patch(rect)
    if show == True:
        ax.set_axis_off()
        plt.tight_layout()
        plt.savefig('foo.png')
        plt.show()
        
    # iterating through the images
    id = 0
    for rectangle in rectangles:
        
        # cropping the images
        top, left, bottom, right = rectangle
        img2 = img.crop((left - padding, top - padding, right + padding, bottom + padding))

        
        img2 = img2.convert('RGB')
        grayPixelsList = []
        
        # converting the image to an array
        data = np.array(img2)
        
        # checking for bad colors
        badColorScore = []
        for colour in colours2:
            if np.all(np.isin(colour, data)) == False:
                badColorScore.append(True)
            else:
                badColorScore.append(False)
        
        
        # checking for necessary colors
        necessaryColorScore = [False]
        if len(colours3) > 0:
            for colour in colours3:
                if np.all(np.isin(colour, data)) == True:
                    necessaryColorScore.append(True)
                else:
                    necessaryColorScore.append(False)
        else:
            necessaryColorScore = [True]
        
        
        if np.all(badColorScore) == True and np.any(necessaryColorScore) == True:
        
            for i in range(0, data.shape[0]):
                grayPixels =  [rgb_to_grayscale(pixel) for pixel in data[i]]
                grayPixelsList.append(grayPixels)
            data_grayscale = np.array(grayPixelsList)

            # converting to black and white
            data_binary, count = convert_binary(data_grayscale, thresh, 0)
            #print('After conversion, there are', count[1][0], 'BLACK pixels and', count[1][1], 'WHITE pixels')
            try:
                imageBlackWhite = Image.fromarray(data_binary.astype(np.uint64))
            except:
                imageBlackWhite = Image.fromarray(data_binary.astype(np.uint8))

            imageBlackWhite = imageBlackWhite.convert('RGB')

            # saving the images
            #print('now saving image', str(id) + ".png")
            id = id + 1
            #imageBlackWhite.save(savePath + "/" + str(id) + ".png")
            
            imageBlackWhite.save(savePath + "/" + str(uuid.uuid1()) + ".png")
                      
        
    return id

In [7]:
beamPurple = (146, (107, 5, 232))
staffGreen = (165, (158, 144, 175))
verticalBeamGreen = (52, (133, 163, 1))
fullBubbleOnLine = (35, (130, 135, 255))
fullBubbleBetweenLines = (37, (56, 0, 0))
emptyBubbleOnLine = (39, (189, 230, 191))
emptyBubbleBetweenLines = (41, (147, 184, 181))
upFlag = (58, (0, 214, 212))
downFlag = (64, (218, 149, 255))

In [14]:
imageFiles = os.listdir('segmentation')
imageFiles = imageFiles[10:12]
print(len(imageFiles))

2


In [15]:
class symbol():
    def __init__(self, name, colorsToTurn, badColours, necessaryColours):
        self.colorsToTurn = colorsToTurn
        self.necessaryColours = necessaryColours
        self.badColours = badColours
        self.necessaryColours = necessaryColours
        self.name = name

In [17]:
symbols = []
symbols.append(symbol('quarterBetweenLines', [fullBubbleBetweenLines[0], verticalBeamGreen[0]], [beamPurple[1], fullBubbleOnLine[1], emptyBubbleOnLine[1], emptyBubbleBetweenLines[1], upFlag[1], downFlag[1]], []))
symbols.append(symbol('quarterOnLine', [fullBubbleOnLine[0], verticalBeamGreen[0]], [beamPurple[1], fullBubbleBetweenLines[1], emptyBubbleOnLine[1], emptyBubbleBetweenLines[1], upFlag[1], downFlag[1]], []))
symbols.append(symbol('eighthBetweenLinesUp', [fullBubbleBetweenLines[0], verticalBeamGreen[0], upFlag[0]], [beamPurple[1], fullBubbleOnLine[1], emptyBubbleOnLine[1], emptyBubbleBetweenLines[1], downFlag[1]], [upFlag[1]]))
symbols.append(symbol('eighthBetweenLinesDown', [fullBubbleBetweenLines[0], verticalBeamGreen[0], downFlag[0]], [beamPurple[1], fullBubbleOnLine[1], emptyBubbleOnLine[1], emptyBubbleBetweenLines[1], upFlag[1]], [downFlag[1]]))
symbols.append(symbol('eighthOnLineUp', [fullBubbleOnLine[0], verticalBeamGreen[0], upFlag[0]], [beamPurple[1], fullBubbleBetweenLines[1], emptyBubbleOnLine[1], emptyBubbleBetweenLines[1], downFlag[1]], [upFlag[1]]))
symbols.append(symbol('eighthOnLineDown', [fullBubbleOnLine[0], verticalBeamGreen[0], downFlag[0]], [beamPurple[1], fullBubbleBetweenLines[0], emptyBubbleOnLine[1], emptyBubbleBetweenLines[1], upFlag[1]], [downFlag[1]]))
symbols.append(symbol('eighthBeamed', [beamPurple[0], fullBubbleBetweenLines[0], fullBubbleOnLine[0], verticalBeamGreen[0]], [emptyBubbleOnLine[1], emptyBubbleBetweenLines[1], upFlag[1], downFlag[1]], [beamPurple[1]]))
symbols.append(symbol('halfBetweenLines', [emptyBubbleBetweenLines[0], verticalBeamGreen[0]], [], [emptyBubbleBetweenLines[1]]))
symbols.append(symbol('halfOnLine', [emptyBubbleOnLine[0], verticalBeamGreen[0]], [], [emptyBubbleOnLine[1]]))


In [18]:
%%time
padding = 30
thresh = 254
show = False
for symbol in symbols[8:9]:
    print('searching for ', symbol.name, '...', sep = '')
    os.makedirs(symbol.name)
    saveTo = symbol.name
    #savedPaths = []
    totalImages = 0
    for imageFile in imageFiles:
        #print('now at image', imageFile) 
        savedImages = save_images('segmentation' + '/' + imageFile, saveTo, symbol.colorsToTurn, symbol.badColours, symbol.necessaryColours, padding, show, thresh)
        #savedPaths.append(symbol.name + '/' + imageFile)
        totalImages = totalImages + savedImages
        print('saved', savedImages, 'images now,', totalImages, 'in total')
        if totalImages > 2000:
            break

    savedPaths = os.listdir(saveTo)
    with ZipFile(symbol.name + '.zip','w') as zip:
        for imagePath in savedPaths:
            zip.write(symbol.name + '/' + imagePath)

    print(totalImages, 'images saved in total')

searching for halfOnLine...
saved 1 images now, 1 in total
saved 5 images now, 6 in total
6 images saved in total
Wall time: 48.2 s
